<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/06_combined_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-19 14:03:46--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11745 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.47K  --.-KB/s    in 0s      

2022-06-19 14:03:46 (87.5 MB/s) - ‘utils.py’ saved [11745/11745]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.3 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 17043 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [ ]:
rgb_input = keras.Input(shape=IMG_SIZE+(3,))
hub_module = hub.KerasLayer(HUB_URL)

rgb_features = hub_module(rgb_input)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = hub_module(xyz_input)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = hub_module(hsv_input)

features = keras.layers.Concatenate()([rgb_features, xyz_features, hsv_features])

output = keras.layers.Dense(N_CLASSES, activation='softmax')(features)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [ ]:
model._name = 'combined-model'

In [ ]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-2, 533, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [12]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='auc', 
                                              verbose=1,
                                              patience=3,
                                              restore_best_weights=True,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name+"-{epoch:02d}.h5")
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='auc',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [ ]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=20,
                    class_weight=class_weight)

Epoch 1/20
533/533 [==============================] - 2728s 5s/step - loss: 356.9871 - auc: 0.4226 - accuracy: 0.5748
Epoch 2/20
533/533 [==============================] - 440s 823ms/step - loss: 184.7210 - auc: 0.5973 - accuracy: 0.7217
Epoch 3/20
533/533 [==============================] - 440s 823ms/step - loss: 160.7567 - auc: 0.6572 - accuracy: 0.7708
Epoch 4/20
533/533 [==============================] - 440s 823ms/step - loss: 131.6319 - auc: 0.7093 - accuracy: 0.8045
Epoch 5/20
533/533 [==============================] - 440s 823ms/step - loss: 108.2154 - auc: 0.7323 - accuracy: 0.8227
Epoch 6/20
533/533 [==============================] - 440s 823ms/step - loss: 79.8795 - auc: 0.7777 - accuracy: 0.8538
Epoch 7/20
533/533 [==============================] - 440s 823ms/step - loss: 75.1899 - auc: 0.7784 - accuracy: 0.8545
Epoch 8/20
533/533 [==============================] - 440s 823ms/step - loss: 54.1386 - auc: 0.8265 - accuracy: 0.8851
Epoch 9/20
533/533 [=========================

In [ ]:
!cp ./logs/checkpoints/combined-model-10.h5 ./drive/MyDrive/checkpoints/combined-model-10.2.h5
!cp ./logs/checkpoints/combined-model-15.h5 ./drive/MyDrive/checkpoints/combined-model-15.2.h5
!cp ./logs/checkpoints/combined-model-20.h5 ./drive/MyDrive/checkpoints/combined-model-20.2.h5

In [13]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=30,
                    initial_epoch=20,
                    class_weight=class_weight)

Epoch 21/30
533/533 [==============================] - 3123s 6s/step - loss: 5.6207 - auc: 0.9592 - accuracy: 0.9698
Epoch 22/30
533/533 [==============================] - 449s 839ms/step - loss: 4.0567 - auc: 0.9673 - accuracy: 0.9739
Epoch 23/30
533/533 [==============================] - 448s 839ms/step - loss: 3.2036 - auc: 0.9746 - accuracy: 0.9780
Epoch 24/30
533/533 [==============================] - 448s 838ms/step - loss: 3.1084 - auc: 0.9736 - accuracy: 0.9782
Epoch 25/30
533/533 [==============================] - 448s 839ms/step - loss: 2.2643 - auc: 0.9796 - accuracy: 0.9810
Epoch 26/30
533/533 [==============================] - 448s 839ms/step - loss: 2.1500 - auc: 0.9825 - accuracy: 0.9830
Epoch 27/30
533/533 [==============================] - 449s 840ms/step - loss: 1.4366 - auc: 0.9879 - accuracy: 0.9886
Epoch 28/30
533/533 [==============================] - 448s 838ms/step - loss: 1.4828 - auc: 0.9863 - accuracy: 0.9868
Epoch 29/30
533/533 [=============================

In [14]:
!cp ./logs/checkpoints/combined-model-25.h5 ./drive/MyDrive/checkpoints/combined-model-25.2.h5
!cp ./logs/checkpoints/combined-model-30.h5 ./drive/MyDrive/checkpoints/combined-model-30.2.h5